# Collision Severity Prediction with Machine Learning Algorithmes - Ramin Ferdos

## Introduction

In this notebook, we are going to analyze histoical data which about accidents and its severities, then with using machine learning algorithmes predicting the severity of newly entered collisions. We are approaching this problem with CRISP-DM Methodologies and further you can find related steps with sections.

---

## First Step - Business Understanding

There are plenty of severity related to coliisons, for example, in a collision severities may be fatality, serious injury, injury, prop damage and etc. but we are reducing this types to two situation: 1. Property only damage collisions and 2. Injury collision.
With appropriate data we can extract valuable knowledge from this prediction we are going to just name a few, for instance, we can understand the probability of injuries in types of collisions for example, rear end collisions result injuries or sidesweeps? also we can mark dangerous spots which the occurance of injury collisions in them are siginifcant, another insight which we can extract is the occurance of collisions especially injury ones in which weathers are more.
so, our problem is predicting severity type of collions based on some features. we will talk about features further.

---

## Second Step - Analytical Approach

Based on the definition of problem, we are going to classify collisions, to two categories. obviously, we need to use classification algorithmes for this problem, there are multiple classifications algorithmes:
- K Nearest Neighbor
- Decision Tree
- Logistic Regression
- Support Vector Machine
- and etc.

It seems like Logistic Regression is a good approach for this problem since our dependent variable (Y) can be just two states (severity of collison type) and also this algorithmes will give us the probablity of each state (i.e for a specific sitations we are predicting that this collison severity will be 89% property damage and 11% injury.)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
